In [1]:
from pymatgen.core import Lattice, Structure, Molecule
from pymatgen.io.cif import CifParser
from pymatgen.io.vasp import Poscar
from pymatgen.io.cif import CifWriter
from pymatgen.io.xyz import XYZ
from pymatgen.io.gaussian import GaussianInput

In [5]:
graphite.apply_strain?

Signature: graphite.apply_strain(strain: 'ArrayLike') -> 'None'
Docstring:
Apply a strain to the lattice.

Args:
    strain (float or list): Amount of strain to apply. Can be a float,
        or a sequence of 3 numbers. E.g., 0.01 means all lattice
        vectors are increased by 1%. This is equivalent to calling
        modify_lattice with a lattice with lattice parameters that
        are 1% larger.
File:      ~/miniconda3/envs/dev/lib/python3.10/site-packages/pymatgen/core/structure.py
Type:      method


In [2]:
strains = [-0.04, -0.03, -0.02, -0.01, 0, 0.01, 0.02, 0.03, 0.04]

In [3]:
structures = {}
for strain in strains:
    graphite = Structure.from_file('graphite.3x3x1.cif')
    graphite.apply_strain([0, 0, strain])
    structures[strain] = graphite

In [4]:
structures[0].sites[0]

PeriodicSite: C (0.0000, 0.0000, 1.6778) [0.0000, 0.0000, 0.2500]

In [5]:
structures[0].sites[0].species_string

'C'

In [6]:
structures[0].sites[0].coords

array([0.     , 0.     , 1.67775])

In [7]:
structures[0].lattice.as_dict()

{'@module': 'pymatgen.core.lattice',
 '@class': 'Lattice',
 'matrix': [[7.392, 0.0, 4.526294569648617e-16],
  [-3.695999999999998, 6.401659784774572, 4.526294569648617e-16],
  [0.0, 0.0, 6.711]]}

In [9]:
structures[-0.04].lattice.as_dict()

{'@module': 'pymatgen.core.lattice',
 '@class': 'Lattice',
 'matrix': [[7.392, 0.0, 4.526294569648617e-16],
  [-3.695999999999998, 6.401659784774572, 4.526294569648617e-16],
  [0.0, 0.0, 6.44256]]}

In [10]:
for i in structures:
    print(structures[i].volume)

304.86882712807585
308.0445440773266
311.2202610265774
314.3959779758282
317.571694925079
320.7474118743298
323.92312882358056
327.0988457728314
330.2745627220822


In [11]:
print('LatticeConstant 1.000000 Ang')
print('%block LatticeVectors')
for i in structures[0].lattice.as_dict()['matrix']:
    print(f"\t{i[0]:.10f}\t{i[1]:.10f}\t{i[2]:.10f}")
print('%endblock LatticeVectors')

LatticeConstant 1.000000 Ang
%block LatticeVectors
	7.3920000000	0.0000000000	0.0000000000
	-3.6960000000	6.4016597848	0.0000000000
	0.0000000000	0.0000000000	6.7110000000
%endblock LatticeVectors


In [12]:
species_index_dict = {
    'C': 1
}
print('AtomicCoordinatesFormat Fractional')
print('%block AtomicCoordinatesAndAtomicSpecies')
for i in structures[0].sites:
    coord = i.coords
    print(f"\t{coord[0]:.9f}\t{coord[1]:.9f}\t{coord[2]:.9f}\t{species_index_dict[i.species_string]}")
print('%endblock AtomicCoordinatesAndAtomicSpecies')

AtomicCoordinatesFormat Fractional
%block AtomicCoordinatesAndAtomicSpecies
	0.000000000	0.000000000	1.677750000	1
	0.000000000	0.000000000	5.033250000	1
	-1.232000000	2.133886595	1.677750000	1
	-2.464000000	4.267773190	5.033250000	1
	2.464000000	4.267773190	1.677750000	1
	1.232000000	2.133886595	5.033250000	1
	2.464000000	0.000000000	1.677750000	1
	4.928000000	0.000000000	5.033250000	1
	2.464000000	0.000000000	5.033250000	1
	4.928000000	0.000000000	1.677750000	1
	2.464000000	4.267773190	5.033250000	1
	1.232000000	2.133886595	1.677750000	1
	-1.232000000	2.133886595	5.033250000	1
	-2.464000000	4.267773190	1.677750000	1
	0.000000000	4.267773190	1.677750000	1
	3.696000000	2.133886595	5.033250000	1
	3.696000000	2.133886595	1.677750000	1
	0.000000000	4.267773190	5.033250000	1
	0.000000000	1.422589641	1.677750000	1
	3.696000000	4.979070144	5.033250000	1
	2.464001232	5.690364964	1.677750000	1
	1.231998768	0.711294820	5.033250000	1
	-2.464001232	5.690364964	1.677750000	1
	6.160001232	0.7112948

In [15]:
len(structures[0])

36

In [16]:
import os

In [23]:
file = "test.fdf"
os.system(f"cat graphite.siesta_template_head.txt > {file}")

with open(file, "a") as f:
    f.write('\n\nLatticeConstant 1.000000 Ang\n')
    f.write('%block LatticeVectors\n')
    for i in structures[0].lattice.as_dict()['matrix']:
        f.write(f"\t{i[0]:.10f}\t{i[1]:.10f}\t{i[2]:.10f}\n")
    f.write('%endblock LatticeVectors\n\n')
    f.write('AtomicCoordinatesFormat Fractional\n')
    f.write('%block AtomicCoordinatesAndAtomicSpecies\n')
    for i in structures[0].sites:
        coord = i.coords
        f.write(f"\t{coord[0]:.9f}\t{coord[1]:.9f}\t{coord[2]:.9f}\t{species_index_dict[i.species_string]}\n")
    f.write('%endblock AtomicCoordinatesAndAtomicSpecies\n\n')

os.system(f"cat graphite.siesta_template_tail.txt >> {file}")

0

In [18]:
%pwd

'/mnt/d/home_base/PHD/ml-potential/pymatgen_test'